In [ ]:
from pyspark.sql.functions import col, trim, when, sum, avg, count
from pyspark.sql.types import IntegerType, DoubleType, StringType

In [ ]:
# Cargar Silver Data
df_transactions_silver = spark.read.parquet("FileStore/silver/credit_transactions")
df_accounts_silver = spark.read.parquet("FileStore/silver/customer_accounts")
df_demographics_silver = spark.read.parquet("FileStore/silver/customer_demographics")
df_merch_categories_silver = spark.read.parquet("FileStore/silver/merchant_categories")

In [ ]:
#GOLD LAYER

#Resumen financiero: Esta tabla contendrá un resumen financiero de cada cliente, combinando información de transacciones y cuentas
#Información: Gasto Total en cada categoría, Promedio de transacciones, Uso de Crédito, Total de transacciones por cliente

customer_financials = df_transactions_silver.groupBy("customer_id").agg(
    sum(col("amount")).alias("total_spent"),
    count(col("transaction_id")).alias("transaction_count"),
    avg(col("amount")).alias("avg_transaction_value")
).join(
    df_accounts_silver.select("customer_id", "credit_limit", "total_due"),
    on="customer_id",
    how="left"
).withColumn(
    "credit_utilization", (col("total_due") / col("credit_limit")) * 100
)

In [ ]:
#Riesgo Crediticio: Tabla para evaluar el nivel de riesgo según comportamiento de pago y uso del crédito.
#Informacion Campos: Clasificación de riesgo bajo, medio o alto.

customer_risk = customer_financials.withColumn(
    "risk_level",
    when((col("credit_utilization") > 80) & (col("total_due") > 5000), "high")
    .when((col("credit_utilization") > 50) & (col("total_due") > 3000), "medium")
    .otherwise("low")
)

In [ ]:
#Consumo por categoría: Esta tabla permite analizar en qué categorías gasta más cada cliente.
#Información Campos: Conteo de transacciones por categoría

customer_spending_by_category = df_transactions_silver.groupBy("customer_id", "merchant_category").agg(
    sum("amount").alias("total_spent_category"),
    count("transaction_id").alias("transaction_count_category")
)

In [ ]:
#Guardar tablas en formato optimizado
customer_financials.write.mode("overwrite").parquet("FileStore/gold/customer_financials")
customer_risk.write.mode("overwrite").parquet("FileStore/gold/customer_risk")
customer_spending_by_category.write.mode("overwrite").parquet("FileStore/gold/customer_spending_by_category")
